## Importing libraries

In [6]:
import pandas
import keras

In [7]:
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential

In [8]:
#from google.colab import drive

In [9]:
#drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
finaldata=pd.read_csv('/content/drive/My Drive/project/data.csv')

In [28]:
finaldata['target']=finaldata['target'].replace({'NoHate':0,'Hate':1})

In [11]:
finaldata.shape

(187902, 11)

In [12]:
finaldata.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'comment_text', 'target', 'Num_words',
       'words', 'clean_comment', 'clean_comment_wo_stop',
       'clean_comment_lemmatized', 'clean_comment_string',
       'clean_comment_string1'],
      dtype='object')

In [13]:
finaldata.sample(2)

,Unnamed: 0.1,Unnamed: 0,comment_text,target,Num_words,words,clean_comment,clean_comment_wo_stop,clean_comment_lemmatized,clean_comment_string,clean_comment_string1
9955,9955,9955,I would check on the Clinton Foundation articl...,NoHate,51,"['I', 'would', 'check', 'on', 'the', 'Clinton'...",i would check on the clinton foundation articl...,would check clinton foundation article edits s...,would check clinton foundation article edits s...,would check clinton foundation article edits s...,would check clinton foundation article edits s...
21775,21775,21775,May 2007 (UTC)\n\nI'm afraid that you've jumpe...,NoHate,30,"['May', '2007', '(UTC)', ""I'm"", 'afraid', 'tha...",may utc i m afraid that you ve jumped ...,may utc afraid jumped without looking although...,may utc afraid jumped without looking although...,may utc afraid jumped without looking although...,may utc afraid jumped without looking although...


In [29]:
textdata=finaldata['clean_comment_string1']
target=finaldata['target']

In [30]:
X_train,X_test,Y_train,Y_test=train_test_split(textdata,target,test_size=0.25,random_state=12,stratify=target)

In [31]:
X_train.shape 

(140926,)

In [32]:
Y_train.value_counts()

0    118563
1     22363
Name: target, dtype: int64

In [25]:
Y_train

128261    NoHate
174406    NoHate
85701     NoHate
24428       Hate
183427      Hate
           ...  
107979    NoHate
180277    NoHate
59923     NoHate
37394     NoHate
93683     NoHate
Name: target, Length: 140926, dtype: object

In [33]:
def tokenizer(X_train, X_test,maxword=75000,sequ=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0).astype('str')
    text = np.array(text)
    tokenizer = Tokenizer(num_words=maxword)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=sequ)
    indices = np.arange(text.shape[0])
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/content/drive/My Drive/project/glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    return (X_train, X_test, word_index,embeddings_index)

In [34]:
train_Glove,test_Glove, wordindex,embeddingsindex = tokenizer(X_train,X_test)

(187902, 50)


In [35]:
def LSTMCNN(wordi, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50):
    kernel_size = 2
    filters = 64
    pool_size = 2
    gru_node = 256
    embedding_matrix = np.random.random((len(wordi) + 1, EMBEDDING_DIM))
    for word, i in wordi.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("EMBEDDING_DIM is not equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector
    model = Sequential()
    model.add(Embedding(len(wordi) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [36]:
modelrcnnlstm = LSTMCNN(wordindex,embeddingsindex, 3)

In [37]:
modelrcnnlstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            8843350   
                                                                 
 dropout_1 (Dropout)         (None, 50, 50)            0         
                                                                 
 conv1d_4 (Conv1D)           (None, 49, 64)            6464      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 24, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_5 (Conv1D)           (None, 23, 64)            8256      
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 11, 64)            0         
 g1D)                                                 

In [38]:
modelrcnnlstm.fit(train_Glove,Y_train,epochs=5,batch_size=128,verbose=2)

Epoch 1/5
1101/1101 - 150s - loss: 0.3166 - accuracy: 0.8750 - 150s/epoch - 136ms/step
Epoch 2/5
1101/1101 - 66s - loss: 0.2586 - accuracy: 0.8953 - 66s/epoch - 60ms/step
Epoch 3/5
1101/1101 - 60s - loss: 0.2431 - accuracy: 0.9009 - 60s/epoch - 54ms/step
Epoch 4/5
1101/1101 - 59s - loss: 0.2294 - accuracy: 0.9058 - 59s/epoch - 54ms/step
Epoch 5/5
1101/1101 - 57s - loss: 0.2170 - accuracy: 0.9113 - 57s/epoch - 52ms/step


In [39]:
hate_prediction=modelrcnnlstm.predict(test_Glove)

1468/1468 [==============================] - 9s 5ms/step


In [40]:
predict_class = np.argmax(hate_prediction, axis=1)

In [41]:
predict_class

array([0, 1, 1, ..., 0, 0, 1])